In [1]:
"""
Ενσωματώνω τα στοιχεία των ζωτικών ενδείξεων
ομαδοποιημένα ανά 8ώρο στους ασθενείς με
εγκεφαλικό που πέρασαν από τα επείγοντα στο
νοσοκομείο.

1.651.119 -> Συνολικές εγγραφές στο vitalsigns.
  752.146 -> Συνολικές κατά την 8ώρη ομαδοποίηση.
    2.645 -> Εγγραφές 8ώρου για τους ασθενείς που
             νοσηλεύτηκαν με εγκεφαλικό από τα
             επείγοντα.
  
"""

'\nΕνσωματώνω τα στοιχεία των ζωτικών ενδείξεων\nομαδοποιημένα ανά 8ώρο στους ασθενείς με\nεγκεφαλικό που πέρασαν από τα επείγοντα στο\nνοσοκομείο.\n\n1.651.119 -> Συνολικές εγγραφές στο vitalsigns.\n  752.146 -> Συνολικές κατά την 8ώρη ομαδοποίηση.\n    2.645 -> Εγγραφές 8ώρου για τους ασθενείς που\n             νοσηλεύτηκαν με εγκεφαλικό από τα\n             επείγοντα.\n  \n'

In [2]:
import pandas as pd

In [3]:
#Εισαγωγή dataframes
df_vitalsign = pd.read_csv('D:\Thesis\mimic-iv-1.0\ed\Vitalsign.csv.gz')
df_ed_patients = pd.read_csv('D:\Thesis\exports\o1_ed_patients_to_hosp.csv',
                             usecols = ['subject_id', 'hadm_id', 'stay_id'])

In [4]:
# Μετατροπή της στήλης charttime από object σε datetime
df_vitalsign['charttime']= pd.to_datetime(df_vitalsign['charttime'])

In [5]:
# Ομαδοποίηση καταγραφών ανά στήλη στο 8ώρο.

grouped_multiple = (df_vitalsign.groupby(['subject_id', 'stay_id',
                                          pd.Grouper(key='charttime',
                                                     freq='8H')]).agg({'temperature': ['mean'],
                                                                       'heartrate': ['mean'],
                                                                       'resprate': ['mean'],
                                                                       'o2sat': ['mean'],
                                                                       'sbp': ['mean'],
                                                                       'dbp': ['mean']}))
grouped_multiple.columns = (['Mean_temperature',
                             'Mean_heartrate',
                             'Mean_resprate',
                             'Mean_o2sat',
                             'Mean_sbp',
                             'Mean_dbp'])
grouped_multiple = grouped_multiple.reset_index()

In [6]:
grouped_multiple

,subject_id,stay_id,charttime,Mean_temperature,Mean_heartrate,Mean_resprate,Mean_o2sat,Mean_sbp,Mean_dbp
0,10000032,32952584,2180-07-22 16:00:00,98.3,84.571429,20.285714,98.0,79.428571,42.857143
1,10000032,33258284,2180-05-06 16:00:00,97.7,79.000000,16.000000,98.0,107.000000,60.000000
2,10000032,35968195,2180-08-05 16:00:00,98.5,96.000000,17.000000,100.0,102.000000,58.000000
3,10000032,35968195,2180-08-06 00:00:00,98.1,91.000000,18.000000,99.0,98.000000,60.000000
4,10000032,38112554,2180-06-26 16:00:00,97.9,81.000000,17.500000,94.0,95.500000,60.500000
...,...,...,...,...,...,...,...,...,...
752141,19999828,30712109,2147-07-18 16:00:00,99.6,78.000000,17.000000,99.0,108.000000,87.000000
752142,19999828,32917002,2149-01-08 08:00:00,97.5,104.400000,16.000000,97.4,120.800000,80.800000
752143,19999828,32917002,2149-01-08 16:00:00,98.1,105.500000,16.000000,96.0,116.000000,74.500000
752144,19999914,32002659,2158-12-24 08:00:00,99.5,81.000000,10.000000,100.0,93.000000,55.000000


In [7]:
# Κενές τιμές στον πίνακα
grouped_multiple.isna().sum()

subject_id              0
stay_id                 0
charttime               0
Mean_temperature    60071
Mean_heartrate       6126
Mean_resprate        7643
Mean_o2sat          28566
Mean_sbp             7394
Mean_dbp             7394
dtype: int64

In [8]:
rslt_df=pd.merge(df_ed_patients, grouped_multiple, on=['stay_id'])

In [9]:
# Διαγραφή της στήλης subject_id_y
rslt_df = rslt_df.drop(['subject_id_y'], axis=1)

# Μετονομασία της στήλης subject_id_x
rslt_df = rslt_df.rename(columns={'subject_id_x': 'subject_id'})

In [10]:
#Εξαγωγή σε csv
rslt_df.to_csv('D:\Thesis\exports\o2_ed_mean_vitals.csv', index=False)